## Deliverable 2. Create a Customer Travel Destinations Map.

In [5]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy 
# Import API key
from config2 import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [6]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Pevek,RU,69.7008,170.3133,-15.93,98,94,7.81,overcast clouds
1,1,Hervey Bay,AU,-25.2986,152.8535,73.00,97,100,5.99,overcast clouds
2,2,Touros,BR,-5.1989,-35.4608,86.00,66,40,14.97,scattered clouds
3,3,Busselton,AU,-33.6500,115.3333,64.00,89,0,1.99,clear sky
4,4,Sitka,US,57.0531,-135.3300,35.60,93,90,8.05,light rain


In [7]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is your desired minimum temperature for your trip? "))
max_temp = float(input("What is your desired maximum temperature for your trip? "))

What is your desired minimum temperature for your trip? 75
What is your desired maximum temperature for your trip? 100


In [8]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Touros,BR,-5.1989,-35.4608,86.00,66,40,14.97,scattered clouds
5,5,Sao Filipe,CV,14.8961,-24.4956,77.29,63,17,1.23,few clouds
6,6,Ladario,BR,-19.0047,-57.6017,89.60,62,20,8.05,few clouds
11,11,Vaini,TO,-21.2000,-175.2000,75.20,83,75,5.75,broken clouds
15,15,Olinda,BR,-8.0089,-34.8553,82.40,65,40,8.05,scattered clouds


In [9]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                6
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [10]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [11]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Touros,BR,86.00,scattered clouds,-5.1989,-35.4608,
5,Sao Filipe,CV,77.29,few clouds,14.8961,-24.4956,
6,Ladario,BR,89.60,few clouds,-19.0047,-57.6017,
11,Vaini,TO,75.20,broken clouds,-21.2000,-175.2000,
15,Olinda,BR,82.40,scattered clouds,-8.0089,-34.8553,
17,Birao,CF,99.37,broken clouds,10.2849,22.7882,
21,Presidencia Roque Saenz Pena,AR,84.99,overcast clouds,-26.7852,-60.4388,
26,Kavieng,PG,77.09,heavy intensity rain,-2.5744,150.7967,
27,Vanimo,PG,78.35,overcast clouds,-2.6741,141.3028,
33,San Policarpo,PH,81.82,broken clouds,12.1791,125.5072,


In [12]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"

    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    
    # 6e. Make request and retrieve the JSON data from the search 
    hotels = requests.get(base_url, params=params).json()
    
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Cannot find hotel ... skipping")
        
        

Cannot find hotel ... skipping
Cannot find hotel ... skipping
Cannot find hotel ... skipping
Cannot find hotel ... skipping
Cannot find hotel ... skipping
Cannot find hotel ... skipping
Cannot find hotel ... skipping
Cannot find hotel ... skipping
Cannot find hotel ... skipping
Cannot find hotel ... skipping


In [15]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Touros,BR,86.00,scattered clouds,-5.1989,-35.4608,INN NEW HORIZON
5,Sao Filipe,CV,77.29,few clouds,14.8961,-24.4956,Tortuga B&B
6,Ladario,BR,89.60,few clouds,-19.0047,-57.6017,Hotel Fazenda 4 cantos
11,Vaini,TO,75.20,broken clouds,-21.2000,-175.2000,Keleti Beach Resort
15,Olinda,BR,82.40,scattered clouds,-8.0089,-34.8553,Hotel Albergue de Olinda


In [16]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [17]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [18]:
# 11a. Add a marker layer for each city to the map. 

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))